### The `__set_name__`  Method

Начиная с Python 3.6, метод `__set_name__` является дополнительным методом, определенным в протоколе дескриптора.

Он вызывается один раз при создании экземпляра дескриптора (то есть при компиляции содержащего его класса) и передает имя свойства в качестве аргумента.

Давайте рассмотрим простой пример, иллюстрирующий это:

In [1]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__ called: owner={owner_class}, prop={property_name}')

In [2]:
class Person:
    name = ValidString()

__set_name__ called: owner=<class '__main__.Person'>, prop=name


Как вы видите, `__set_name__` был вызван при создании класса `Person`. Это единственный раз, когда он вызывается.

Главное преимущество этого в том, что мы можем захватить имя свойства:

In [3]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__ called: owner={owner_class}, prop={property_name}')
        self.property_name = property_name

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            print(f'__get__ called for property {self.property_name} '
                  f'of instance {instance}')

In [4]:
class Person:
    first_name = ValidString()
    last_name = ValidString()

__set_name__ called: owner=<class '__main__.Person'>, prop=first_name
__set_name__ called: owner=<class '__main__.Person'>, prop=last_name


Теперь посмотрите, что происходит, когда мы получаем свойство из экземпляров:

In [5]:
p = Person()

In [6]:
p.first_name

__get__ called for property first_name of instance <__main__.Person object at 0x7fa4604f3cf8>


In [7]:
p.last_name

__get__ called for property last_name of instance <__main__.Person object at 0x7fa4604f3cf8>


Итак, по сути, мы знаем, какое имя свойства было назначено экземпляру дескриптора.

Это может быть удобно для сообщений, которые могут ссылаться на имя свойства, или даже для хранения значений в словаре экземпляра (предполагая, что мы можем):

In [8]:
class ValidString():
    def __init__(self, min_length):
        self.min_length = min_length

    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be a string.')
        if len(value) < self.min_length:
            raise ValueError(f'{self.property_name} must be at least '
                             f'{self.min_length} characters'
                            )
        key = '_' + self.property_name
        setattr(instance, key, value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            key = '_' + self.property_name
            return getattr(instance, key, None)

In [9]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [10]:
p = Person()

In [11]:
try:
    p.first_name = 'Alex'
    p.last_name = 'M'
except ValueError as ex:
    print(ex)

last_name must be at least 2 characters


Приятно знать, что `last_name` — это свойство, вызывающее исключение!

Мы также использовали имя свойства в качестве основы для атрибута в самом экземпляре:

In [12]:
p = Person()
p.first_name = 'Alex'

In [13]:
p.first_name, p.__dict__

('Alex', {'_first_name': 'Alex'})

Итак, хотя это теперь устраняет проблему, которую мы видели в начале этого раздела (когда пользователь дважды указывает имя свойства), у нас все еще есть проблема потенциальной перезаписи существующего атрибута экземпляра:

In [14]:
p = Person()

In [15]:
p._first_name = 'some data I need to store'

In [16]:
p.__dict__

{'_first_name': 'some data I need to store'}

In [17]:
p.first_name = 'Alex'

In [18]:
p.__dict__

{'_first_name': 'Alex'}

Это привело к уничтожению наших данных. Это нехорошо, и нам нужно что-то с этим делать.

А как насчет сохранения значения в экземпляре с использованием того же имени?

Вспомните, как атрибуты экземпляра затеняют атрибуты класса:

In [19]:
class BankAccount:
    apr = 10

In [20]:
b = BankAccount()

In [21]:
b.apr, b.__dict__

(10, {})

In [22]:
b.apr = 20

In [23]:
b.apr, b.__dict__

(20, {'apr': 20})

Итак, как вы видите, дескриптор — это атрибут **класса**. Так что если мы сохраним значение под тем же именем в экземпляре, не столкнемся ли мы с этой проблемой затенения, когда атрибут теперь будет использовать атрибут в экземпляре, а не атрибут дескриптора класса?

И ответ — это зависит!

Данные и не данные дескрипторы — это различие важно, и мы рассмотрим его в следующих лекциях.

Давайте быстро рассмотрим это:

In [24]:
class ValidString:
    def __init__(self, min_length):
        self.min_length = min_length

    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be a string.')
        if len(value) < self.min_length:
            raise ValueError(f'{self.property_name} must be at least '
                             f'{self.min_length} characters'
                            )
        instance.__dict__[self.property_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            print (f'calling __get__ for {self.property_name}')
            return instance.__dict__.get(self.property_name, None)

In [25]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [26]:
p = Person()

In [27]:
p.__dict__

{}

In [28]:
p.first_name = 'Alex'

In [29]:
p.__dict__

{'first_name': 'Alex'}

Итак, `first_name` находится в словаре экземпляра, и мы ожидаем, что доступ к `first_name` будет использовать словарь экземпляра:

In [30]:
p.first_name

calling __get__ for first_name


'Alex'

Ага, он использовал дескриптор!!

Давайте рассмотрим это подробнее далее.